In [13]:
from tqdm import tqdm
import pandas as pd
import torch
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [14]:
data = pd.read_csv('environments/guivans_folder/data-2 (1).csv')
# data = pd.read_csv('data-2 (1)-2.csv')
data

,video,text,extraversion,neuroticism,agreeableness,interview,openness,conscientiousness,text_preprocessed
0,J4GQm9j0JZ0.003.mp4,He's cutting it and then turn around and see t...,0.523364,0.552083,0.626374,0.504673,0.488889,0.601942,hes cutting turn around see end result im glad...
1,zEyRyTnIw5I.005.mp4,Responsibility to house the organ I had been g...,0.345794,0.375000,0.472527,0.457944,0.366667,0.582524,responsibility house organ given needed tell g...
2,nskJh7v6v1U.004.mp4,I actually got quite a few sets of black pens ...,0.252336,0.291667,0.406593,0.373832,0.511111,0.485437,actually got quite sets black pens year bought...
3,6wHQsN5g2RM.000.mp4,I ate a lot. I'd like a lot of foods. I rememb...,0.457944,0.489583,0.505495,0.457944,0.377778,0.398058,ate lot id like lot foods remember favorite ma...
4,dQOeQYWIgm8.000.mp4,Now I'll ask you guys to leave a question in t...,0.607477,0.489583,0.406593,0.570093,0.622222,0.621359,ill ask guys leave question comments hopefully...
...,...,...,...,...,...,...,...,...,...
5995,Eh7WRYXVh9M.000.mp4,"I got to see a lot, got to experience a lot. Y...",0.523364,0.479167,0.626374,0.588785,0.544444,0.621359,got see lot got experience lot yeah didnt know...
5996,2q8orkMs2Jg.003.mp4,"Really, really love my job, love the people I ...",0.728972,0.760417,0.582418,0.616822,0.822222,0.524272,really really love job love people work feel f...
5997,F1lAPYh4t3U.000.mp4,I have a nine-to-five. I said this in my last ...,0.700935,0.770833,0.747253,0.691589,0.788889,0.699029,ninetofive said last qa ninetofive per se prob...
5998,cxJ0u6r0-pU.001.mp4,You want to be working with and who you want t...,0.317757,0.531250,0.582418,0.616822,0.588889,0.679612,want working want help bring anyone could get ...


In [15]:
data_valid = pd.read_csv('environments/data_valid.csv')
# data_valid = pd.read_csv('data_valid.csv')
data_valid

,video,text,extraversion,neuroticism,agreeableness,interview,openness,conscientiousness,text_preprocessed
0,modNfUPt3F4.002.mp4,[inaudible 00:00:16] and then I will do the vo...,0.644860,0.593750,0.615385,0.616822,0.555556,0.640777,inaudible 000016 voice ill shots thats talking...
1,h6LOjpCRXtY.005.mp4,"Personality is A1, I like you. So that is all ...",0.439252,0.520833,0.417582,0.439252,0.411111,0.572816,personality a1 like questions four minutes tod...
2,WER4ww680QQ.004.mp4,"Ewe. But yeah, some people been asking me if I...",0.457944,0.312500,0.428571,0.373832,0.555556,0.398058,ewe yeah people asking still like one directio...
3,c4XnKouozXU.002.mp4,"I am a makeup artist, I am an aesthetician, I ...",0.364486,0.572917,0.527473,0.523364,0.322222,0.553398,makeup artist aesthetician someone also extrem...
4,OEKg-Tvwcbk.002.mp4,"Sucks. When he agrees, yeah. It just sucks bec...",0.345794,0.468750,0.516484,0.383178,0.477778,0.417476,sucks agrees yeah sucks hot cant go walks girl...
...,...,...,...,...,...,...,...,...,...
1995,3LAaFUSGvsU.000.mp4,Really qualified to. The guy who last week was...,0.570093,0.614583,0.494505,0.626168,0.577778,0.689320,really qualified guy last week accountant week...
1996,n2BuwHbdilY.000.mp4,"Wearing scarfs on the second date. Okay, so no...",0.542056,0.541667,0.549451,0.579439,0.666667,0.669903,wearing scarfs second date okay hats scarfs pl...
1997,GcuoyJPO-KU.003.mp4,"""How do you manage to do things with someone? ...",0.551402,0.593750,0.560440,0.504673,0.644444,0.572816,manage things someone got used ive shapes acry...
1998,uf_sIIw4zxY.004.mp4,"-I sell. No. Sophie Loveland, ""Will you be pos...",0.514019,0.552083,0.461538,0.551402,0.733333,0.572816,sell sophie loveland posting video wedding you...


In [16]:
def map_range(x, a, b, c, d):
    return c + (d - c) * (x - a) / (b - a)

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
tokenizer = BertTokenizer.from_pretrained("Minej/bert-base-personality")
model = BertForSequenceClassification.from_pretrained("Minej/bert-base-personality").to(device)

def personality_detection(text):
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    outputs = model(**inputs)
    predictions = outputs.logits.squeeze().detach().numpy()

    label_names = ['Extroversion', 'Neuroticism', 'Agreeableness', 'Conscientiousness', 'Openness']
    result = {label_names[i]: predictions[i] for i in range(len(label_names))}

    return result

# Train

In [104]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.squeeze()
    mse = mean_squared_error(labels, preds)
    return {'MSES': mse}

In [105]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",            # Совпадает с evaluation_strategy
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=16,
    weight_decay=0.01,
    no_cuda=False,
    load_best_model_at_end=True,      # Загрузка лучшей модели по окончании
    metric_for_best_model="MSES",     # Метрика, по которой выбирается лучшая модель
    greater_is_better=False           # Установите в False, если метрика должна минимизироваться
)

/home/user1/environments/hack/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [106]:
class PersonalityDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=85):
        self.tokenizer = tokenizer
        self.texts = dataframe['text'].map(str).tolist()
        
        # Получение 5 целевых значений для каждого примера
        self.targets = dataframe[['extraversion', 'openness', 'conscientiousness', 'agreeableness', 'neuroticism']].values
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        target = self.targets[idx]

        # Токенизация
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(target, dtype=torch.float)  # Возвращаем 5-мерный тензор
        }

In [107]:
train_dataset = PersonalityDataset(data, tokenizer)
test_dataset = PersonalityDataset(data_valid, tokenizer)

In [108]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [109]:
trainer.train()

Epoch,Training Loss,Validation Loss,Mses
1,0.690800,0.690781,0.281930
2,0.690600,0.689938,0.177870
3,0.690500,0.690090,0.253548
4,0.690700,0.689790,0.212183
5,0.695100,0.698528,0.055683
6,0.690400,0.689979,0.208937
7,0.690300,0.689873,0.196855
8,0.690600,0.689910,0.235000
9,0.689900,0.689742,0.214472
10,0.689900,0.689670,0.210290


TrainOutput(global_step=24000, training_loss=0.690584706624349, metrics={'train_runtime': 1266.2592, 'train_samples_per_second': 75.814, 'train_steps_per_second': 18.953, 'total_flos': 4193445395520000.0, 'train_loss': 0.690584706624349, 'epoch': 16.0})

In [110]:
# trainer.save_model("./best_model")

In [20]:
def predict_personality_traits(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits.squeeze().cpu().numpy()  # Переводим на CPU и в numpy

    traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]
    return dict(zip(traits, predictions))

In [19]:
model = BertForSequenceClassification.from_pretrained("./best_model").to(device)

In [21]:
predict_personality_traits(data.iloc[50].text)

{'Openness': 0.70907116,
 'Conscientiousness': 1.2218435,
 'Extraversion': 1.0589265,
 'Agreeableness': 0.9691025,
 'Neuroticism': 1.2008816}

In [22]:
preds = []

for i, row in tqdm(data_valid.iterrows()):
    preds.append(predict_personality_traits(str(row.text)))

0it [00:00, ?it/s]

2000it [00:19, 104.21it/s]


In [23]:
preds = pd.DataFrame(preds)
preds

,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism
0,0.296347,0.690376,0.417417,0.472202,0.417897
1,-0.171134,0.258618,0.247354,0.335115,0.238763
2,-0.210255,0.305415,-0.008714,0.257146,0.145233
3,0.326545,0.839618,0.514829,0.650687,0.672809
4,0.316596,0.661830,0.388892,0.571474,0.508637
...,...,...,...,...,...
1995,-0.141296,0.433503,0.398665,0.555429,0.308167
1996,-0.288181,0.220752,0.192603,0.255032,-0.040319
1997,-0.531018,-0.005694,-0.137100,0.011455,-0.220955
1998,0.028742,0.482168,0.310379,0.424124,0.370328


In [33]:
s = 0
s1 = 0
s2 = 0

for i in ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']:
    a = preds[i].min()
    b = preds[i].max()
    c = mean_squared_error(data_valid[i.lower()], preds[i].map(lambda x: map_range(x, a, b, 0, 1)))
    d = c ** 0.5
    e = mean_absolute_error(data_valid[i.lower()], preds[i].map(lambda x: map_range(x, a, b, 0, 1)))
    print(i, "MSE:", c, "RMSE:", d, "MAE:", e, a, b)
    s += c
    s1 += d
    s2 += e

print("===============================================================================================")
print("MSE:", s, "RMSE:", s1, "MAE:", s2)

Openness MSE: 0.038638479393160834 RMSE: 0.1965667301278648 MAE: 0.1568962927850261 -1.156436 0.89340734
Conscientiousness MSE: 0.04564503655432543 RMSE: 0.21364699051080835 MAE: 0.17245063755491813 -0.6558855 1.3051943
Extraversion MSE: 0.04243137137840986 RMSE: 0.20598876517521497 MAE: 0.16446037527513813 -0.7169265 1.2328148
Agreeableness MSE: 0.038106205998554826 RMSE: 0.1952081094589946 MAE: 0.15564002225845536 -0.4751214 1.0410889
Neuroticism MSE: 0.051474445789826774 RMSE: 0.22687980472009134 MAE: 0.1836191304961734 -0.90752506 1.1106781
MSE: 0.2162955391142777 RMSE: 1.038290399992974 MAE: 0.8330664583697112
